In [ ]:
#@title
# Upload: 
# mnist_train_regression.csv
# mnist_test_regression.csv
# mnist_train_classification.csv
# mnist_test_classification.csv

# 6.5. Convolutional Neural Networks (CNN)

In [ ]:
#@title 6.5.1. GPU runtime and importing some necessary packages
import numpy as np
import pandas as pd

import sklearn.preprocessing as pg
import keras.utils           as ku 

import matplotlib
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models

#from tensorflow.keras.layers import Dense
#from sklearn.preprocessing import MinMaxScaler

In [ ]:
#@title 6.5.2. Regression example: data preprocessing
data_tr   = pd.read_csv('/content/mnist_train_regression.csv')
data_te   = pd.read_csv('/content/mnist_test_regression.csv')

datain_tr = data_tr.iloc[:,:-1]
dataou_tr = data_tr.iloc[:,-1:]

datain_te = data_te.iloc[:,:-1]
dataou_te = data_te.iloc[:,-1:]

print("training inputs: \n\n {} \n\n".format(datain_tr))
print("testing  inputs: \n\n {} \n\n".format(datain_te))

# get the values
datain_tr = datain_tr.values
dataou_tr = dataou_tr.values

datain_te = datain_te.values
dataou_te = dataou_te.values

# output calibration (inputs are calibrated row-wised between 0 and 1)

fun_calibration_ou = pg.MinMaxScaler(feature_range=(0,1))
fun_calibration_ou.fit(dataou_tr)

dataou_tr_calibrated = fun_calibration_ou.transform(dataou_tr)
dataou_te_calibrated = fun_calibration_ou.transform(dataou_te)

# reshape for CNN layers

shape0 = datain_tr.shape[0]
shape1 = int(np.sqrt(datain_tr.shape[1]))
shape2 = int(np.sqrt(datain_tr.shape[1]))
shape3 = 1

datain_tr = datain_tr.reshape((shape0, shape1, shape2, shape3))
datain_te = datain_te.reshape((shape0, shape1, shape2, shape3))

# observe some training data
plt.figure(figsize=[10,10])
fig, axs = plt.subplots(3, 3)
counter = 0
for i0 in range(3):
  for i1 in range(3):
    axs[i0,i1].imshow(datain_tr[counter,:,:,0].transpose(), cmap=plt.get_cmap('gray'))
    counter = counter +1

In [ ]:
#@title 6.5.3. Regression example: create and compile a CNN model
model = models.Sequential()

model.add(layers.Conv2D(filters = 8, kernel_size=(3, 3), strides=(1, 1), input_shape=(shape1, shape2, shape3)))
model.add(layers.LayerNormalization())
model.add(layers.ReLU())

model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(layers.Conv2D(filters = 16, kernel_size=(3, 3), strides=(1, 1)))
model.add(layers.LayerNormalization())
model.add(layers.ReLU())

model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(layers.Conv2D(filters = 32, kernel_size=(3, 3), strides=(1, 1)))
model.add(layers.LayerNormalization())
model.add(layers.ReLU())


model.add(layers.Flatten())
model.add(layers.Dropout(rate = 0.1))

model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dropout(rate = 0.1))
model.add(layers.Dense(50 , activation='relu'))
model.add(layers.Dropout(rate = 0.1))
model.add(layers.Dense(25 , activation='relu'))
model.add(layers.Dropout(rate = 0.1))
model.add(layers.Dense(1  , activation='linear'))

model.compile(optimizer='adam', loss = "mean_squared_error")

model.summary()


In [ ]:
#@title 6.5.4. Regression example: train and test the CNN model
history = model.fit(datain_tr, dataou_tr_calibrated, epochs = 100, batch_size = 128, verbose = 1, shuffle=True, validation_split=0.01)

dataes_tr_calibrated = model.predict(datain_tr)
dataes_te_calibrated = model.predict(datain_te)

In [ ]:
#@title 6.5.5. Regression example: plots
plt.figure(figsize=[5,5])
plt.plot(dataou_tr_calibrated[:,0], dataes_tr_calibrated[:,0], '.', markersize = 3)
plt.plot([0,1],[0,1], '-r', linewidth = 2)
plt.xlabel('Real', fontsize = 20)
plt.ylabel('Estimated', fontsize = 20)
plt.title('Training Results | CNN', fontsize = 20)
plt.legend(['datapoint','bi-sector'], fontsize = 20)
matplotlib.rc('xtick', labelsize = 20)
matplotlib.rc('ytick', labelsize = 20)

plt.figure(figsize=[5,5])
plt.plot(dataou_te_calibrated[:,0], dataes_te_calibrated[:,0], '.', markersize = 3)
plt.plot([0,1],[0,1], '-r', linewidth = 2)
plt.xlabel('Real', fontsize = 20)
plt.ylabel('Estimated', fontsize = 20)
plt.title('Testing Results | CNN', fontsize = 20)
plt.legend(['datapoint','bi-sector'], fontsize = 20)
matplotlib.rc('xtick', labelsize = 20)
matplotlib.rc('ytick', labelsize = 20)

plt.figure(figsize=[8,8])
plt.plot(history.history['loss'],'--')
plt.plot(history.history['val_loss'],'-')
plt.title('model loss', fontsize = 20)
plt.ylabel('loss', fontsize = 20)
plt.xlabel('epoch', fontsize = 20)
plt.legend(['training loss','validation loss'], fontsize = 20)
plt.show()

In [ ]:
#@title 6.5.6. Classification example: data preprocessing
data_tr   = pd.read_csv('/content/mnist_train_classification.csv')
data_te   = pd.read_csv('/content/mnist_test_classification.csv')

datain_tr = data_tr.iloc[:,:-1]
dataou_tr = data_tr.iloc[:,-1:]

datain_te = data_te.iloc[:,:-1]
dataou_te = data_te.iloc[:,-1:]

print("training inputs: \n\n {} \n\n".format(datain_tr))
print("testing  inputs: \n\n {} \n\n".format(datain_te))

# get the values
datain_tr = datain_tr.values
dataou_tr = dataou_tr.values

datain_te = datain_te.values
dataou_te = dataou_te.values

# No calibration (inputs are calibrated row-wised and outputs are "classes")

# output to categorical
dataou_tr_categorical = ku.to_categorical(dataou_tr)
dataou_te_categorical = ku.to_categorical(dataou_te)


# reshape for CNN layers

shape0 = datain_tr.shape[0]
shape1 = int(np.sqrt(datain_tr.shape[1]))
shape2 = int(np.sqrt(datain_tr.shape[1]))
shape3 = 1

datain_tr = datain_tr.reshape((shape0, shape1, shape2, shape3))
datain_te = datain_te.reshape((shape0, shape1, shape2, shape3))

# observe some training data
plt.figure(figsize=[10,10])
fig, axs = plt.subplots(3, 3)
counter = 0
for i0 in range(3):
  for i1 in range(3):
    axs[i0,i1].imshow(datain_tr[counter,:,:,0].transpose(), cmap=plt.get_cmap('gray'))
    counter = counter +1

In [ ]:
#@title 6.5.7. Classification example: create and compile a CNN model
model = models.Sequential()

model.add(layers.Conv2D(filters = 8, kernel_size=(3, 3), strides=(1, 1), input_shape=(shape1, shape2, shape3)))
model.add(layers.LayerNormalization())
model.add(layers.ReLU())

model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(layers.Conv2D(filters = 16, kernel_size=(3, 3), strides=(1, 1)))
model.add(layers.LayerNormalization())
model.add(layers.ReLU())

model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(layers.Conv2D(filters = 32, kernel_size=(3, 3), strides=(1, 1)))
model.add(layers.LayerNormalization())
model.add(layers.ReLU())


model.add(layers.Flatten())
model.add(layers.Dropout(rate = 0.1))

model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dropout(rate = 0.1))
model.add(layers.Dense(50 , activation='relu'))
model.add(layers.Dropout(rate = 0.1))
model.add(layers.Dense(25 , activation='relu'))
model.add(layers.Dropout(rate = 0.1))
model.add(layers.Dense(10  , activation='softmax'))

model.compile(optimizer='adam', loss = "categorical_crossentropy", metrics=['accuracy'])

model.summary()


In [ ]:
#@title 6.5.8. Classification example: train and test the CNN model
history = model.fit(datain_tr, dataou_tr_categorical, epochs = 100, batch_size = 128, verbose = 1, shuffle=True, validation_split=0.01)

dataes_tr_categorical = model.predict(datain_tr)
dataes_te_categorical = model.predict(datain_te)

In [ ]:
#@title 6.5.9. Classification example: compute accuracy
def fun_accuracy(dataes, dataou):
  num_err = np.count_nonzero( dataes.argmax(axis = 1) - dataou.argmax(axis = 1) )
  accuracy = 1 - num_err/dataou.shape[0]
  return accuracy 

print('training accuracy: {} %'.format( fun_accuracy(dataes_tr_categorical,dataou_tr_categorical)*100) )
print('testing  accuracy: {} %'.format( fun_accuracy(dataes_te_categorical,dataou_te_categorical)*100) )



In [ ]:
#@title 6.5.10. Classification example: plots
plt.figure(figsize=[8,8])
plt.plot(history.history['loss'],'--')
plt.plot(history.history['val_loss'],'-')
plt.title('model loss', fontsize = 20)
plt.ylabel('loss', fontsize = 20)
plt.xlabel('epoch', fontsize = 20)
plt.legend(['training loss','validation loss'], fontsize = 20)
plt.show()

plt.figure(figsize=[8,8])
plt.plot(history.history['accuracy'],'--')
plt.plot(history.history['val_accuracy'],'-')
plt.title('model accuracy', fontsize = 20)
plt.ylabel('accuracy', fontsize = 20)
plt.xlabel('epoch', fontsize = 20)
plt.legend(['training accuracy','validation accuracy'], fontsize = 20)
plt.show()